## Part 4 - Deploy Model

**Connect to Azure ML workspace**

In [ ]:
from azureml.core import Workspace

workspace = Workspace.from_config()

**Get model**

In [ ]:
from azureml.core import Model

model = Model(workspace, name='pokemon-classifier')

**Define inference config**

In [ ]:
%pycat scoring/score.py

In [ ]:
%pycat scoring/env.yml

In [ ]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(source_directory='scoring',
                                   runtime='python', 
                                   entry_script='score.py',
                                   conda_file='env.yml')

**Deploy web service**

In [ ]:
from azureml.core.webservice import Webservice
from azureml.exceptions import WebserviceException
from azureml.core.webservice import AciWebservice, Webservice

aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=4)
aci_service_name = 'pokemon-classifier'

try:
    aci_service = Webservice(workspace, name=aci_service_name)
    if aci_service:
        aci_service.delete()
except WebserviceException as e:
    print()

aci_service = Model.deploy(workspace, aci_service_name, [model], inference_config, aci_config)
aci_service.wait_for_deployment(True)

**Test web service**

In [ ]:
from azureml.core.webservice import AciWebservice

aci_service = AciWebservice(workspace, 'pokemon-classifier')

In [ ]:
service_url = aci_service.scoring_uri

In [ ]:
# Load classes
classes = []
f = open('training/classes.txt','r')
for line in f.readlines():
    classes.append(line.replace('\n',''))
    f.close()

In [ ]:
import ipywidgets as widgets
from ipywidgets import Layout, Button, Box, Label, Text, VBox
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
from matplotlib import rcParams
%matplotlib inline

def on_button_clicked(b):
    with output:
        image_url = label.children[1].value
        
        # Make HTTP call
        input_data = '{\"image_url\": \"'+ image_url +'\"}'
        headers = {'Content-Type':'application/json'}
        response = requests.post(service_url, input_data, headers=headers)
        
        # Download image
        image = Image.open(BytesIO(requests.get(image_url).content))

        # Get prediction
        result = np.array(json.loads(json.loads(response.text))['prediction'][0])
        prediction = 'Prediction: {}\n Probability: {}'.format(classes[result.argmax()], result.max())

        # Display image
        plt.title(prediction)
        plt.imshow(image)
        plt.show()
        
output = widgets.Output()

label = Box([Label(value='Pokemon Image URL: '), Text()])
button = Button(description='Get Prediction!')
button.on_click(on_button_clicked)

form = Box([label, button], layout=Layout(
    display='flex',
    flex_flow='row',
    align_items='stretch',
    width='100%'
))

In [ ]:
'''
Examples: 
https://github.com/johnwu0604/mcgill-aml-workshop/blob/master/test-images/pikachu_toy.png?raw=true
https://github.com/johnwu0604/mcgill-aml-workshop/blob/master/test-images/charmander_counter.png?raw=true
https://github.com/johnwu0604/mcgill-aml-workshop/blob/master/test-images/charmander_hidden.png?raw=true
https://github.com/johnwu0604/mcgill-aml-workshop/blob/master/test-images/bulbasaur_plush.png?raw=true
https://github.com/johnwu0604/mcgill-aml-workshop/blob/master/test-images/mewtwo_toy.png?raw=true
https://github.com/johnwu0604/mcgill-aml-workshop/blob/master/test-images/squirtle_plush.png?raw=true
'''
VBox([form, output])

In [ ]:
output.clear_output()